In [33]:
from __future__ import print_function

from keras.models import Sequential, Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Activation, Dense, Permute, Dropout, add, dot,merge, concatenate
from keras.layers import LSTM
from keras.utils.data_utils import get_file
from keras.preprocessing.sequence import pad_sequences
from functools import reduce
import tarfile
import numpy as np
import re

In [34]:
def tokenize(sent):
    # splitting the sentences based on spaces and tokenizing them
    return [x.strip() for x in re.split('(\W+)?', sent) if x.strip()]



In [35]:
def parse_stories(lines, only_supporting=False):
   
    # Trying to retrive the questions and 
    # ansers to the particular questions
    
    data = []
    story = []
    for line in lines:
        line = line.decode('utf-8').strip()
        nid, line = line.split(' ', 1)
        nid = int(nid)
        if nid == 1:
            story = []
        if '\t' in line:
            q, a, supporting = line.split('\t')
            q = tokenize(q)
            substory = None
            if only_supporting:
                # Only select the related substory
                supporting = map(int, supporting.split())
                substory = [story[i - 1] for i in supporting]
            else:
                # Provide all the substories
                substory = [x for x in story if x]
            data.append((substory, q, a))
            story.append('')
        else:
            sent = tokenize(line)
            story.append(sent)
        
    return data


In [36]:
def get_stories(f, only_supporting=False, max_length=None):
    
    # Discaring the stories longer than the max length defined 
    # and converting the lines in to a single story corresponding 
    # to the questions
    data = parse_stories(f.readlines(), only_supporting=only_supporting)
    flatten = lambda data: reduce(lambda x, y: x + y, data)
    data = [(flatten(story), q, answer) for story, q, answer in data if not max_length or len(flatten(story)) < max_length]
    return data


In [37]:
def vectorize_stories(data, word_idx, story_maxlen, query_maxlen):

    # Mapping the length of stories against story with maximum
    # length and doing necessary padding
    
    X = []
    Xq = []
    Y = []
    for story, query, answer in data:
        x=[word_idx[w] for w in story]
        xq=[word_idx[w] for w in query]
        y=np.zeros(len(word_idx) + 1)
        y[word_idx[answer]]=1
        X.append(x)
        Xq.append(xq)
        Y.append(y)
    return (pad_sequences(X, maxlen=story_maxlen),
            pad_sequences(Xq, maxlen=query_maxlen),
            np.array(Y))


In [38]:
''' Accessing the particula modules of the 
Babi dataset saperated by the modules
'''

try:
    path = get_file('babi-tasks-v1-2.tar.gz', origin='https://s3.amazonaws.com/text-datasets/babi_tasks_1-20_v1-2.tar.gz')
except:
    print('Error downloading dataset, please download it manually:\n'
          '$ wget http://www.thespermwhale.com/jaseweston/babi/tasks_1-20_v1-2.tar.gz\n'
          '$ mv tasks_1-20_v1-2.tar.gz ~/.keras/datasets/babi-tasks-v1-2.tar.gz')
    raise
tar = tarfile.open(path)


In [39]:
challenges = {'qa4_two-arg-relations': 'tasks_1-20_v1-2/en-10k/qa4_two-arg-relations_{}.txt'}
challenge_type = 'qa4_two-arg-relations'
challenge = challenges[challenge_type]

print('Extracting stories for the challenge:', challenge_type)
train_stories = get_stories(tar.extractfile(challenge.format('train')))
test_stories = get_stories(tar.extractfile(challenge.format('test')))

Extracting stories for the challenge: qa4_two-arg-relations


/usr/lib/python3.5/re.py:203: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [40]:
print(train_stories)

[(['The', 'office', 'is', 'north', 'of', 'the', 'kitchen', '.', 'The', 'garden', 'is', 'south', 'of', 'the', 'kitchen', '.'], ['What', 'is', 'north', 'of', 'the', 'kitchen', '?'], 'office'), (['The', 'kitchen', 'is', 'west', 'of', 'the', 'garden', '.', 'The', 'hallway', 'is', 'west', 'of', 'the', 'kitchen', '.'], ['What', 'is', 'the', 'garden', 'east', 'of', '?'], 'kitchen'), (['The', 'garden', 'is', 'north', 'of', 'the', 'office', '.', 'The', 'bedroom', 'is', 'north', 'of', 'the', 'garden', '.'], ['What', 'is', 'north', 'of', 'the', 'garden', '?'], 'bedroom'), (['The', 'bedroom', 'is', 'west', 'of', 'the', 'hallway', '.', 'The', 'kitchen', 'is', 'west', 'of', 'the', 'bedroom', '.'], ['What', 'is', 'the', 'hallway', 'east', 'of', '?'], 'bedroom'), (['The', 'bathroom', 'is', 'east', 'of', 'the', 'bedroom', '.', 'The', 'kitchen', 'is', 'west', 'of', 'the', 'bedroom', '.'], ['What', 'is', 'the', 'bedroom', 'east', 'of', '?'], 'kitchen'), (['The', 'bedroom', 'is', 'south', 'of', 'the', 'of

In [41]:
print(test_stories)

[(['The', 'hallway', 'is', 'east', 'of', 'the', 'bathroom', '.', 'The', 'bedroom', 'is', 'west', 'of', 'the', 'bathroom', '.'], ['What', 'is', 'the', 'bathroom', 'east', 'of', '?'], 'bedroom'), (['The', 'bedroom', 'is', 'west', 'of', 'the', 'kitchen', '.', 'The', 'hallway', 'is', 'west', 'of', 'the', 'bedroom', '.'], ['What', 'is', 'west', 'of', 'the', 'kitchen', '?'], 'bedroom'), (['The', 'bathroom', 'is', 'north', 'of', 'the', 'garden', '.', 'The', 'hallway', 'is', 'north', 'of', 'the', 'bathroom', '.'], ['What', 'is', 'north', 'of', 'the', 'garden', '?'], 'bathroom'), (['The', 'kitchen', 'is', 'south', 'of', 'the', 'bathroom', '.', 'The', 'bedroom', 'is', 'south', 'of', 'the', 'kitchen', '.'], ['What', 'is', 'south', 'of', 'the', 'kitchen', '?'], 'bedroom'), (['The', 'bedroom', 'is', 'east', 'of', 'the', 'kitchen', '.', 'The', 'office', 'is', 'east', 'of', 'the', 'bedroom', '.'], ['What', 'is', 'east', 'of', 'the', 'kitchen', '?'], 'bedroom'), (['The', 'garden', 'is', 'east', 'of', 

In [42]:
vocab = set()
for story, q, answer in train_stories + test_stories:
    print(story)
    vocab |= set(story + q + [answer])
vocab = sorted(vocab)

# Reserve 0 for masking via pad_sequences
vocab_size = len(vocab) + 1
story_maxlen = max(map(len, (x for x, _, _ in train_stories + test_stories)))
query_maxlen = max(map(len, (x for _, x, _ in train_stories + test_stories)))

['The', 'office', 'is', 'north', 'of', 'the', 'kitchen', '.', 'The', 'garden', 'is', 'south', 'of', 'the', 'kitchen', '.']
['The', 'kitchen', 'is', 'west', 'of', 'the', 'garden', '.', 'The', 'hallway', 'is', 'west', 'of', 'the', 'kitchen', '.']
['The', 'garden', 'is', 'north', 'of', 'the', 'office', '.', 'The', 'bedroom', 'is', 'north', 'of', 'the', 'garden', '.']
['The', 'bedroom', 'is', 'west', 'of', 'the', 'hallway', '.', 'The', 'kitchen', 'is', 'west', 'of', 'the', 'bedroom', '.']
['The', 'bathroom', 'is', 'east', 'of', 'the', 'bedroom', '.', 'The', 'kitchen', 'is', 'west', 'of', 'the', 'bedroom', '.']
['The', 'bedroom', 'is', 'south', 'of', 'the', 'office', '.', 'The', 'bedroom', 'is', 'north', 'of', 'the', 'garden', '.']
['The', 'garden', 'is', 'west', 'of', 'the', 'office', '.', 'The', 'bedroom', 'is', 'west', 'of', 'the', 'garden', '.']
['The', 'kitchen', 'is', 'east', 'of', 'the', 'hallway', '.', 'The', 'bathroom', 'is', 'west', 'of', 'the', 'hallway', '.']
['The', 'office', '

['The', 'bedroom', 'is', 'north', 'of', 'the', 'garden', '.', 'The', 'office', 'is', 'south', 'of', 'the', 'garden', '.']
['The', 'kitchen', 'is', 'west', 'of', 'the', 'office', '.', 'The', 'garden', 'is', 'east', 'of', 'the', 'office', '.']
['The', 'garden', 'is', 'east', 'of', 'the', 'bedroom', '.', 'The', 'kitchen', 'is', 'west', 'of', 'the', 'bedroom', '.']
['The', 'office', 'is', 'north', 'of', 'the', 'bathroom', '.', 'The', 'office', 'is', 'south', 'of', 'the', 'hallway', '.']
['The', 'kitchen', 'is', 'west', 'of', 'the', 'hallway', '.', 'The', 'hallway', 'is', 'west', 'of', 'the', 'bedroom', '.']
['The', 'hallway', 'is', 'west', 'of', 'the', 'bathroom', '.', 'The', 'kitchen', 'is', 'east', 'of', 'the', 'bathroom', '.']
['The', 'kitchen', 'is', 'north', 'of', 'the', 'bathroom', '.', 'The', 'bedroom', 'is', 'north', 'of', 'the', 'kitchen', '.']
['The', 'kitchen', 'is', 'west', 'of', 'the', 'bedroom', '.', 'The', 'kitchen', 'is', 'east', 'of', 'the', 'garden', '.']
['The', 'garden'

['The', 'bathroom', 'is', 'north', 'of', 'the', 'kitchen', '.', 'The', 'kitchen', 'is', 'north', 'of', 'the', 'bedroom', '.']
['The', 'bathroom', 'is', 'north', 'of', 'the', 'hallway', '.', 'The', 'office', 'is', 'south', 'of', 'the', 'hallway', '.']
['The', 'kitchen', 'is', 'east', 'of', 'the', 'hallway', '.', 'The', 'bedroom', 'is', 'east', 'of', 'the', 'kitchen', '.']
['The', 'garden', 'is', 'south', 'of', 'the', 'hallway', '.', 'The', 'office', 'is', 'north', 'of', 'the', 'hallway', '.']
['The', 'hallway', 'is', 'south', 'of', 'the', 'garden', '.', 'The', 'kitchen', 'is', 'south', 'of', 'the', 'hallway', '.']
['The', 'garden', 'is', 'south', 'of', 'the', 'bedroom', '.', 'The', 'hallway', 'is', 'north', 'of', 'the', 'bedroom', '.']
['The', 'bathroom', 'is', 'west', 'of', 'the', 'bedroom', '.', 'The', 'office', 'is', 'west', 'of', 'the', 'bathroom', '.']
['The', 'kitchen', 'is', 'south', 'of', 'the', 'bathroom', '.', 'The', 'office', 'is', 'south', 'of', 'the', 'kitchen', '.']
['The'

['The', 'office', 'is', 'east', 'of', 'the', 'hallway', '.', 'The', 'bathroom', 'is', 'east', 'of', 'the', 'office', '.']
['The', 'hallway', 'is', 'south', 'of', 'the', 'bathroom', '.', 'The', 'garden', 'is', 'north', 'of', 'the', 'bathroom', '.']
['The', 'hallway', 'is', 'west', 'of', 'the', 'bathroom', '.', 'The', 'office', 'is', 'east', 'of', 'the', 'bathroom', '.']
['The', 'garden', 'is', 'north', 'of', 'the', 'bathroom', '.', 'The', 'bedroom', 'is', 'south', 'of', 'the', 'bathroom', '.']
['The', 'bedroom', 'is', 'east', 'of', 'the', 'bathroom', '.', 'The', 'bathroom', 'is', 'east', 'of', 'the', 'office', '.']
['The', 'kitchen', 'is', 'north', 'of', 'the', 'office', '.', 'The', 'bathroom', 'is', 'south', 'of', 'the', 'office', '.']
['The', 'bedroom', 'is', 'north', 'of', 'the', 'kitchen', '.', 'The', 'kitchen', 'is', 'north', 'of', 'the', 'garden', '.']
['The', 'office', 'is', 'south', 'of', 'the', 'hallway', '.', 'The', 'kitchen', 'is', 'north', 'of', 'the', 'hallway', '.']
['The'

['The', 'hallway', 'is', 'west', 'of', 'the', 'bedroom', '.', 'The', 'garden', 'is', 'east', 'of', 'the', 'bedroom', '.']
['The', 'office', 'is', 'east', 'of', 'the', 'bedroom', '.', 'The', 'bathroom', 'is', 'west', 'of', 'the', 'bedroom', '.']
['The', 'hallway', 'is', 'east', 'of', 'the', 'bedroom', '.', 'The', 'garden', 'is', 'east', 'of', 'the', 'hallway', '.']
['The', 'kitchen', 'is', 'south', 'of', 'the', 'hallway', '.', 'The', 'bedroom', 'is', 'south', 'of', 'the', 'kitchen', '.']
['The', 'kitchen', 'is', 'east', 'of', 'the', 'bathroom', '.', 'The', 'office', 'is', 'west', 'of', 'the', 'bathroom', '.']
['The', 'garden', 'is', 'west', 'of', 'the', 'office', '.', 'The', 'bathroom', 'is', 'west', 'of', 'the', 'garden', '.']
['The', 'office', 'is', 'north', 'of', 'the', 'bedroom', '.', 'The', 'bedroom', 'is', 'north', 'of', 'the', 'garden', '.']
['The', 'hallway', 'is', 'east', 'of', 'the', 'bedroom', '.', 'The', 'office', 'is', 'west', 'of', 'the', 'bedroom', '.']
['The', 'bedroom',

['The', 'hallway', 'is', 'north', 'of', 'the', 'kitchen', '.', 'The', 'hallway', 'is', 'south', 'of', 'the', 'bedroom', '.']
['The', 'garden', 'is', 'west', 'of', 'the', 'hallway', '.', 'The', 'kitchen', 'is', 'east', 'of', 'the', 'hallway', '.']
['The', 'bedroom', 'is', 'east', 'of', 'the', 'bathroom', '.', 'The', 'bedroom', 'is', 'west', 'of', 'the', 'hallway', '.']
['The', 'office', 'is', 'east', 'of', 'the', 'hallway', '.', 'The', 'office', 'is', 'west', 'of', 'the', 'garden', '.']
['The', 'office', 'is', 'west', 'of', 'the', 'bathroom', '.', 'The', 'garden', 'is', 'west', 'of', 'the', 'office', '.']
['The', 'bedroom', 'is', 'west', 'of', 'the', 'hallway', '.', 'The', 'bathroom', 'is', 'east', 'of', 'the', 'hallway', '.']
['The', 'kitchen', 'is', 'east', 'of', 'the', 'hallway', '.', 'The', 'kitchen', 'is', 'west', 'of', 'the', 'office', '.']
['The', 'bedroom', 'is', 'south', 'of', 'the', 'garden', '.', 'The', 'hallway', 'is', 'south', 'of', 'the', 'bedroom', '.']
['The', 'kitchen',

['The', 'office', 'is', 'south', 'of', 'the', 'bedroom', '.', 'The', 'hallway', 'is', 'north', 'of', 'the', 'bedroom', '.']
['The', 'office', 'is', 'west', 'of', 'the', 'bathroom', '.', 'The', 'office', 'is', 'east', 'of', 'the', 'hallway', '.']
['The', 'hallway', 'is', 'south', 'of', 'the', 'office', '.', 'The', 'kitchen', 'is', 'south', 'of', 'the', 'hallway', '.']
['The', 'bathroom', 'is', 'north', 'of', 'the', 'bedroom', '.', 'The', 'bathroom', 'is', 'south', 'of', 'the', 'office', '.']
['The', 'bedroom', 'is', 'south', 'of', 'the', 'bathroom', '.', 'The', 'garden', 'is', 'south', 'of', 'the', 'bedroom', '.']
['The', 'hallway', 'is', 'north', 'of', 'the', 'garden', '.', 'The', 'garden', 'is', 'north', 'of', 'the', 'bedroom', '.']
['The', 'garden', 'is', 'north', 'of', 'the', 'kitchen', '.', 'The', 'kitchen', 'is', 'north', 'of', 'the', 'office', '.']
['The', 'bedroom', 'is', 'south', 'of', 'the', 'hallway', '.', 'The', 'garden', 'is', 'north', 'of', 'the', 'hallway', '.']
['The', '

In [54]:
with open('story_maxlen_two-arg-relations.txt','w') as file:
    file.write(str(story_maxlen))
    file.write('\n')
file.close()


with open('query_maxlen_two-arg-relations.txt','w') as file:
    file.write(str(query_maxlen))
    file.write('\n')
file.close()


In [44]:
''' Knowing some facts and stats about the 
module of dataset we are using
'''

print('-')
print('Vocab size:', vocab_size, 'unique words')
print('Story max length:', story_maxlen, 'words')
print('Query max length:', query_maxlen, 'words')
print('Number of training stories:', len(train_stories))
print('Number of test stories:', len(test_stories))
print('-')
print('Here\'s what a "story" tuple looks like (input, query, answer):')
print(train_stories[0])
print('-')
print('Vectorizing the word sequences...')

word_idx = dict((c, i + 1) for i, c in enumerate(vocab))
idx_word = dict((i+1, c) for i,c in enumerate(vocab))
inputs_train, queries_train, answers_train = vectorize_stories(train_stories,
                                                               word_idx,
                                                               story_maxlen,
                                                               query_maxlen)
inputs_test, queries_test, answers_test = vectorize_stories(test_stories,
                                                            word_idx,
                                                            story_maxlen,
                                                            query_maxlen)

-
Vocab size: 18 unique words
Story max length: 16 words
Query max length: 7 words
Number of training stories: 10000
Number of test stories: 1000
-
Here's what a "story" tuple looks like (input, query, answer):
(['The', 'office', 'is', 'north', 'of', 'the', 'kitchen', '.', 'The', 'garden', 'is', 'south', 'of', 'the', 'kitchen', '.'], ['What', 'is', 'north', 'of', 'the', 'kitchen', '?'], 'office')
-
Vectorizing the word sequences...


In [45]:
with open('word_idx_two-arg-relations.txt','w') as file:
    file.write(str(word_idx))
file.close()

In [46]:
with open('idx_word_two-arg-relations.txt','w') as file:
    file.write(str(idx_word))
file.close()

In [47]:
print(word_idx)

{'bedroom': 6, 'kitchen': 11, 'The': 3, 'office': 14, '.': 1, 'garden': 8, 'bathroom': 5, 'hallway': 9, 'What': 4, 'the': 16, 'north': 12, 'west': 17, 'is': 10, 'south': 15, 'of': 13, '?': 2, 'east': 7}


In [48]:
print(idx_word)

{1: '.', 2: '?', 3: 'The', 4: 'What', 5: 'bathroom', 6: 'bedroom', 7: 'east', 8: 'garden', 9: 'hallway', 10: 'is', 11: 'kitchen', 12: 'north', 13: 'of', 14: 'office', 15: 'south', 16: 'the', 17: 'west'}


In [49]:
print('-')
print('inputs: integer tensor of shape (samples, max_length)')
print('inputs_train shape:', inputs_train.shape)
print('inputs_test shape:', inputs_test.shape)
print('-')
print('queries: integer tensor of shape (samples, max_length)')
print('queries_train shape:', queries_train.shape)
print('queries_test shape:', queries_test.shape)
print('-')
print('answers: binary (1 or 0) tensor of shape (samples, vocab_size)')
print('answers_train shape:', answers_train.shape)
print('answers_test shape:', answers_test.shape)
print('-')
print('Compiling...')

-
inputs: integer tensor of shape (samples, max_length)
inputs_train shape: (10000, 16)
inputs_test shape: (1000, 16)
-
queries: integer tensor of shape (samples, max_length)
queries_train shape: (10000, 7)
queries_test shape: (1000, 7)
-
answers: binary (1 or 0) tensor of shape (samples, vocab_size)
answers_train shape: (10000, 18)
answers_test shape: (1000, 18)
-
Compiling...


In [50]:
train_epochs = 120
batch_size = 32
lstm_size = 64

In [51]:
# initiating 
input_sequence = Input((story_maxlen,))
question = Input((query_maxlen,))

print('Input sequence:', input_sequence)
print('Question:', question)

# defining the encoder for the input and embedding the vocab
input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_size,
                              output_dim=64))
input_encoder_m.add(Dropout(0.3))

# mapping the dimensions according to the maximum length on input
input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_size,
                              output_dim=query_maxlen))
input_encoder_c.add(Dropout(0.3))

# vectorizing the questions
question_encoder = Sequential()
question_encoder.add(Embedding(input_dim=vocab_size,
                               output_dim=64,
                               input_length=query_maxlen))
question_encoder.add(Dropout(0.3))


# conctinating input stories and questions
input_encoded_m = input_encoder_m(input_sequence)
print('Input encoded m', input_encoded_m)
input_encoded_c = input_encoder_c(input_sequence)
print('Input encoded c', input_encoded_c)
question_encoded = question_encoder(question)
print('Question encoded', question_encoded)


# cross refrencing the questions against the stories
match = merge([input_encoded_m, question_encoded], mode='dot', dot_axes=(2, 2))
print(match.shape)
match = Activation('softmax')(match)
print('Match shape', match)

response = merge([match, input_encoded_c], mode='sum')
response = Permute((2, 1))(response)  
print('Response shape', response)

# mapping answers against the vector of question and stories
answer = merge([response, question_encoded], mode='concat')
print('Answer shape', answer)

answer = LSTM(64)(answer)  
answer = Dropout(0.3)(answer)
answer = Dense(vocab_size)(answer)  

# popping out prob. of answers
answer = Activation('softmax')(answer)

Input sequence: Tensor("input_3:0", shape=(?, 16), dtype=float32)
Question: Tensor("input_4:0", shape=(?, 7), dtype=float32)
Input encoded m Tensor("sequential_7/dropout_5/cond/Merge:0", shape=(?, 16, 64), dtype=float32)
Input encoded c Tensor("sequential_8/dropout_6/cond/Merge:0", shape=(?, 16, 7), dtype=float32)
Question encoded Tensor("sequential_9/dropout_7/cond/Merge:0", shape=(?, 7, 64), dtype=float32)
(?, 16, 7)
Match shape Tensor("activation_3/truediv:0", shape=(?, 16, 7), dtype=float32)
Response shape Tensor("permute_2/transpose:0", shape=(?, 7, 16), dtype=float32)
Answer shape Tensor("merge_6/concat:0", shape=(?, 7, 80), dtype=float32)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:38: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/home/ubuntu/src/keras/keras/legacy/layers.py:464: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:43: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:48: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [52]:
model = Model([input_sequence, question], answer)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])

In [53]:
model.fit([inputs_train, queries_train], answers_train, batch_size, train_epochs,
          validation_data=([inputs_test, queries_test], answers_test))

Train on 10000 samples, validate on 1000 samples
Epoch 1/120
10000/10000 [==============================] - 6s 562us/step - loss: 1.7886 - acc: 0.2248 - val_loss: 1.4129 - val_acc: 0.4090
Epoch 2/120
10000/10000 [==============================] - 5s 507us/step - loss: 1.3021 - acc: 0.4564 - val_loss: 1.1456 - val_acc: 0.4770
Epoch 3/120
10000/10000 [==============================] - 5s 503us/step - loss: 1.1574 - acc: 0.5129 - val_loss: 1.0463 - val_acc: 0.5300
Epoch 4/120
10000/10000 [==============================] - 5s 502us/step - loss: 1.0355 - acc: 0.5608 - val_loss: 0.8579 - val_acc: 0.5760
Epoch 5/120
10000/10000 [==============================] - 5s 502us/step - loss: 0.9152 - acc: 0.6067 - val_loss: 0.7172 - val_acc: 0.6500
Epoch 6/120
10000/10000 [==============================] - 5s 503us/step - loss: 0.8620 - acc: 0.6232 - val_loss: 0.6797 - val_acc: 0.6810
Epoch 7/120
10000/10000 [==============================] - 5s 502us/step - loss: 0.8085 - acc: 0.6566 - val_loss: 0.6

10000/10000 [==============================] - 5s 502us/step - loss: 0.0110 - acc: 0.9966 - val_loss: 3.5599e-06 - val_acc: 1.0000
Epoch 117/120
10000/10000 [==============================] - 5s 501us/step - loss: 0.0089 - acc: 0.9978 - val_loss: 3.7161e-06 - val_acc: 1.0000
Epoch 118/120
10000/10000 [==============================] - 5s 503us/step - loss: 0.0088 - acc: 0.9981 - val_loss: 1.8795e-06 - val_acc: 1.0000
Epoch 119/120
10000/10000 [==============================] - 5s 503us/step - loss: 0.0081 - acc: 0.9982 - val_loss: 5.1065e-06 - val_acc: 1.0000
Epoch 120/120
10000/10000 [==============================] - 5s 503us/step - loss: 0.0050 - acc: 0.9988 - val_loss: 2.2165e-06 - val_acc: 1.0000


In [55]:
''' Getting the feel for the test dataset
and how it is devided.
'''

' Getting the feel for the test dataset\nand how it is devided.\n'

In [56]:
for i in range(0,10):
        current_inp = test_stories[i]
        current_story, current_query, current_answer = vectorize_stories([current_inp], word_idx, story_maxlen, query_maxlen)
        current_prediction = model.predict([current_story, current_query])
        current_prediction = idx_word[np.argmax(current_prediction)]
        print(' '.join(current_inp[0]), ' '.join(current_inp[1]), '| Prediction:', current_prediction, '| Ground Truth:', current_inp[2])
        print("-----------------------------------------------------------------------------------------")
        

The hallway is east of the bathroom . The bedroom is west of the bathroom . What is the bathroom east of ? | Prediction: bedroom | Ground Truth: bedroom
-----------------------------------------------------------------------------------------
The bedroom is west of the kitchen . The hallway is west of the bedroom . What is west of the kitchen ? | Prediction: bedroom | Ground Truth: bedroom
-----------------------------------------------------------------------------------------
The bathroom is north of the garden . The hallway is north of the bathroom . What is north of the garden ? | Prediction: bathroom | Ground Truth: bathroom
-----------------------------------------------------------------------------------------
The kitchen is south of the bathroom . The bedroom is south of the kitchen . What is south of the kitchen ? | Prediction: bedroom | Ground Truth: bedroom
-----------------------------------------------------------------------------------------
The bedroom is east of the k

In [57]:
print(story)

['The', 'garden', 'is', 'west', 'of', 'the', 'hallway', '.', 'The', 'garden', 'is', 'east', 'of', 'the', 'kitchen', '.']


In [58]:
from keras.models import model_from_json
model_json = model.to_json()
with open("model_two-arg-relations.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_two-arg-relations.h5")
print("Saved model to disk")

Saved model to disk


In [59]:
json_file = open('model_two-arg-relations.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model_two-arg-relations.h5")

/home/ubuntu/src/keras/keras/engine/topology.py:1269: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  return cls(**config)


In [60]:
test_stories[0]

(['The',
  'hallway',
  'is',
  'east',
  'of',
  'the',
  'bathroom',
  '.',
  'The',
  'bedroom',
  'is',
  'west',
  'of',
  'the',
  'bathroom',
  '.'],
 ['What', 'is', 'the', 'bathroom', 'east', 'of', '?'],
 'bedroom')

In [61]:
loaded_model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
for i in range(0,10):
        current_inp = test_stories[i]
        current_story, current_query, current_answer = vectorize_stories([current_inp], word_idx, story_maxlen, query_maxlen)
        current_prediction = loaded_model.predict([current_story, current_query])
        current_prediction = idx_word[np.argmax(current_prediction)]
        print(' '.join(current_inp[0]), ' '.join(current_inp[1]), '| Prediction:', current_prediction, '| Ground Truth:', current_inp[2])
        print("-----------------------------------------------------------------------------------------")


The hallway is east of the bathroom . The bedroom is west of the bathroom . What is the bathroom east of ? | Prediction: bedroom | Ground Truth: bedroom
-----------------------------------------------------------------------------------------
The bedroom is west of the kitchen . The hallway is west of the bedroom . What is west of the kitchen ? | Prediction: bedroom | Ground Truth: bedroom
-----------------------------------------------------------------------------------------
The bathroom is north of the garden . The hallway is north of the bathroom . What is north of the garden ? | Prediction: bathroom | Ground Truth: bathroom
-----------------------------------------------------------------------------------------
The kitchen is south of the bathroom . The bedroom is south of the kitchen . What is south of the kitchen ? | Prediction: bedroom | Ground Truth: bedroom
-----------------------------------------------------------------------------------------
The bedroom is east of the k

In [62]:
print(current_story)

[[ 3 14 10  7 13 16  9  1  3  9 10  7 13 16  8  1]]


In [63]:
print(current_query)

[[ 4 10  7 13 16  9  2]]


In [64]:
print(len(current_query))

1


In [65]:
print(test_stories[0])

(['The', 'hallway', 'is', 'east', 'of', 'the', 'bathroom', '.', 'The', 'bedroom', 'is', 'west', 'of', 'the', 'bathroom', '.'], ['What', 'is', 'the', 'bathroom', 'east', 'of', '?'], 'bedroom')
